In [20]:
# Lista de Exercícios de Mineração de Dados
# Gustavo Nunes Lopes

In [21]:
# Questão 1
# A)

import pathlib
import numpy as np
import pandas as pd

def remove_incomplete_data(data: list):
    """
    Remover dados faltantes (-100) de uma lista de listas.
    Só adiciona à lista se todos os valores forem válidos.
    """
    cleaned_data = []
    for i in range(len(data)):
        for j in range(len(data[i])):
            if data[i][j] == -100:
                break
        else:
            cleaned_data.append(data[i])

    return cleaned_data

def split_X_y(data: list):
    """
    Separa os dados em atributos (X) e rótulos (y).
    """
    X = np.array([row[:7] for row in data], dtype=float)
    y = np.array([row[7] for row in data])  # label

    return X, y

def accuracy_metric(actual, predicted):
    """
    Calcula a acurácia entre os valores reais e previstos.
    """
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1

    return correct / len(actual) * 100.0

def rocchio_train(X_train, y_train):
    """
    Treina o classificador Rocchio calculando os centróides para cada classe.
    """
    centroids = {}
    classes = np.unique(y_train)

    for c in classes:
        Xc = X_train[y_train == c]
        centroids[c] = np.mean(Xc, axis=0)  # vetor de 7 dimensões
    return centroids

def rocchio_predict(centroids, X_test):
    """ 
    Faz previsões usando o classificador Rocchio.
    """   
    classes = list(centroids.keys())
    y_pred = []

    for x in X_test:
        best_class = None
        best_dist = None

        for c in classes:
            d = euclidean_distance(x, centroids[c])
            if best_dist is None or d < best_dist:
                best_dist = d
                best_class = c

        y_pred.append(best_class)

    return np.array(y_pred)

def euclidean_distance(point1, point2):
    """
    Calcula a distância euclidiana entre dois pontos. 
    """
    return np.sqrt(np.sum((point1 - point2) ** 2))

def nn_predict(X_train, y_train, X_test):
    """ 
    Faz previsões usando o classificador Nearest Neighbor (NN).
    """
    y_pred = []

    for x in X_test:
        best_dist = None
        best_label = None

        for i in range(len(X_train)):
            d = euclidean_distance(x, X_train[i])

            if best_dist is None or d < best_dist:
                best_dist = d
                best_label = y_train[i]

        y_pred.append(best_label)

    return np.array(y_pred)

# Carregando os arquivos de texto
nebulosa_path_train = pathlib.Path().absolute() / 'db' / 'nebulosa_train.txt'
nebulosa_path_test = pathlib.Path().absolute() / 'db' / 'nebulosa_test.txt'

# Lendo os arquivos
with open(nebulosa_path_train, 'r') as file:
    nebulosa_train = file.read().split()
    new_nebulosa_train = []
    for i in range(0, len(nebulosa_train), 8):
        new_nebulosa_train.append(nebulosa_train[i:i+8])

    for i in range(len(new_nebulosa_train)):
        for j in range(len(new_nebulosa_train[i])):
            if j != 7:
                new_nebulosa_train[i][j] = float(new_nebulosa_train[i][j])


# Main
with open(nebulosa_path_test, 'r') as file:
    nebulosa_test = file.read().split()
    new_nebulosa_test = []
    for i in range(0, len(nebulosa_test), 8):
        new_nebulosa_test.append(nebulosa_test[i:i+8])

    for i in range(len(new_nebulosa_test)):
        for j in range(len(new_nebulosa_test[i])):
            if j != 7:
                new_nebulosa_test[i][j] = float(new_nebulosa_test[i][j])

nebulosa_train_cleaned = remove_incomplete_data(new_nebulosa_train)
nebulosa_test_cleaned = remove_incomplete_data(new_nebulosa_test)
    
print("Treino original:", len(new_nebulosa_train), " | Treino limpo:", len(nebulosa_train_cleaned))
print("Teste original :", len(new_nebulosa_test),  " | Teste limpo :", len(nebulosa_test_cleaned))

X_train, y_train = split_X_y(nebulosa_train_cleaned)
X_test, y_test   = split_X_y(nebulosa_test_cleaned)

print("\n===== QUESTÃO 1A =====")

# ---- NN ----
y_pred_nn = nn_predict(X_train, y_train, X_test)
acc_nn = accuracy_metric(y_test, y_pred_nn)
print(f"Acurácia NN: {acc_nn:.2f}%")

# ---- Rocchio ----
centroids = rocchio_train(X_train, y_train)
y_pred_rocchio = rocchio_predict(centroids, X_test)
acc_rocchio = accuracy_metric(y_test, y_pred_rocchio)
print(f"Acurácia Rocchio: {acc_rocchio:.2f}%")

Treino original: 143  | Treino limpo: 134
Teste original : 28  | Teste limpo : 25

===== QUESTÃO 1A =====
Acurácia NN: 56.00%
Acurácia Rocchio: 0.00%


In [22]:
# Questão 1
# B)

def clip_outliers_iqr(X_train, X_test, k=1.5):
    """
    Para remover outliers usando o método do IQR.
    Referência: https://www.geeksforgeeks.org/data-science/detect-and-remove-the-outliers-using-python/
    """
    Q1 = np.percentile(X_train, 25, axis=0, method='midpoint')
    Q3 = np.percentile(X_train, 75, axis=0, method='midpoint')
    IQR = Q3 - Q1

    lower = Q1 - k * IQR
    upper = Q3 + k * IQR

    X_train_clipped = np.clip(X_train, lower, upper)
    X_test_clipped  = np.clip(X_test,  lower, upper)

    return X_train_clipped, X_test_clipped

def minmax_normalize(X_train, X_test):
    """ 
    Normaliza os dados usando a técnica Min-Max.
    """
    min_val = np.min(X_train, axis=0)
    max_val = np.max(X_train, axis=0)

    X_train_scaled = (X_train - min_val) / (max_val - min_val)
    X_test_scaled  = (X_test  - min_val) / (max_val - min_val)

    return X_train_scaled, X_test_scaled

X_train, y_train = split_X_y(nebulosa_train_cleaned)
X_test, y_test   = split_X_y(nebulosa_test_cleaned)

# Main

print("===== QUESTÃO 1B =====")

# Outliers
X_train_clp, X_test_clp = clip_outliers_iqr(X_train, X_test)

# Min–Max
X_train_mm, X_test_mm = minmax_normalize(X_train_clp, X_test_clp)

# ---- NN ----
y_pred_nn_mm = nn_predict(X_train_mm, y_train, X_test_mm)
acc_nn_mm = accuracy_metric(y_test, y_pred_nn_mm)
print(f"Acurácia NN: {acc_nn_mm:.2f}%")

# ---- Rocchio ----
centroids_mm = rocchio_train(X_train_mm, y_train)
y_pred_rocchio_mm = rocchio_predict(centroids_mm, X_test_mm)
acc_rocchio_mm = accuracy_metric(y_test, y_pred_rocchio_mm)
print(f"Acurácia Rocchio: {acc_rocchio_mm:.2f}%")

===== QUESTÃO 1B =====
Acurácia NN: 64.00%
Acurácia Rocchio: 4.00%


In [23]:
# Questão 1
# C)

# Os resultados obtidos na questão 1B mostram que uma aplicação de técnicas de pré-processamento ajudam a melhorar a acurácia dos classificadores NN e Rocchio.
# Isso se deve ao fato de que a remoção de outliers e a utilização de valores normalizados reduz o impacto de valores extremos que podem distorcer a análise dos dados.

In [24]:
# Questão 2
# A)

import spacy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

tweets_path = pathlib.Path().absolute() / 'db' / 'Tweets_Mg.csv'
df = pd.read_csv(tweets_path)

nlp = spacy.load("pt_core_news_sm")

def embeddings_generator(textos):
    embeddings = []
    for doc in nlp.pipe(textos, batch_size=50):
        embeddings.append(doc.vector)
    return np.array(embeddings)

print("===== QUESTÃO 2A =====")

X = embeddings_generator(df["Text"].astype(str))
y = df["Classificacao"].values

X_train = X[:8000]
y_train = y[:8000]

X_test = X[8000:]
y_test = y[8000:]

y_pred = nn_predict(X_train, y_train, X_test)

accuracy = accuracy_metric(y_test, y_pred)
print("Acurácia NN (implementação própria): ", accuracy)

# Agora utilizando o scikit-learn
knn = KNeighborsClassifier(n_neighbors=1) # k=5 por padrão, então pra ficar igual à nossa implementação 
# usamos k=1

knn.fit(X_train, y_train)
y_pred_sklearn = knn.predict(X_test)

accuracy_scikit_learn = accuracy_score(y_test, y_pred_sklearn) * 100

print("Acurácia NN (scikit-learn):", accuracy_scikit_learn)

===== QUESTÃO 2A =====
Acurácia NN (implementação própria):  89.9497487437186
Acurácia NN (scikit-learn): 89.9497487437186


In [25]:
# Questão 2
# A) (Continuação)

# A acurácia obtida utilizando a implementaçlão manual é igual a utilizando 
# a biblioteca scikit-learn, pois ambos os métodos utilizam o mesmo algoritmo. A diferença
# é que a minha implementação manual foi feita pra utilizar apenas k=1, enquanto a biblioteca
# scikit-learn permite escolher o valor de k.

In [ ]:
# Questão 2
# B)

In [27]:
# Questão 3
# A)

In [28]:
# Questão 3
# B)

In [29]:
# Questão 3
# C)

In [30]:
# Questão 4
# A)

In [31]:
# Questão 4
# B)

In [32]:
# Questão 4
# C)

In [33]:
# Questão 4
# D)

In [34]:
# Questão 5
# A)

In [35]:
# Questão 5
# B)

In [36]:
# Questão 5
# C)

In [37]:
# Questão 6
# A)

In [38]:
# Questão 6
# B)

In [39]:
# Questão 6
# C)

In [40]:
# Questão 7
# A)

In [41]:
# Questão 7
# B)

In [42]:
# Questão 7
# C)

In [ ]:
# Questão 8
 # Não estou conseguindo acessar o link

In [44]:
# Questão 9

In [45]:
##################### QUESTÕES TEÓRICAS #####################

In [46]:
# Questão 1
# a) “Quanto mais variáveis de entrada forem usadas em um modelo de aprendizado de 
# máquina, melhor será a qualidade do modelo”. 

# Resposta: Depende, pois, embora mais variáveis possam fornecer mais informações, 
# elas também podem conter muitos dados ruidosos, faltantes, ou até mesmo dados irrelevantes, 
# oque pode fazer com que o modelo aprenda padrões incorretos (overfitting). 


In [47]:
# Questão 1
# b) “Independente da qualidade, quanto mais amostras forem obtidas para uma base de 
# dados, maior a tendência de se obter modelos mais adequados”.

# Resposta: Mentira, pois, apesar de mais amostras geralmente ajudarem a melhorar a performance do modelo,
# a qualidade dos dados é crucial, pois, dados de baixa qualidade podem introduzir ruído.

In [48]:
# Questão 1
# c) “Às vezes com simples manipulações na base de dados (limpeza, conversão de valores, 
# etc.) pode-se conseguir melhoras significativas nos resultados, sem fazer nenhuma 
# alteração na técnica de aprendizado de máquina usada”. 

# Resposta: Verdade, pois, técnicas de pré-processamento de dados pode melhorar a qualidade dos dados,
# o que pode levar a melhores resultados sem alterar o modelo.

In [49]:
# Questão 2

# Investimento conservador
# Valor Esperado = 30000*0,1 + 5000*0,5 + −10000*0,4
# Valor Esperado = 3000 + 2500 − 4000 = 1500

# Investimento especulativo
# Valor Esperado = 40000*0,1 + 10000*0,5 + −30000*0,4
# Valor Esperado = 4000 + 5000 − 12000 = -3000

# Investimento cíclico
# Valor Esperado = -10000*0,1 + 0*0,5 + 15000*0,4
# Valor Esperado = -1000 + 0 + 6000 = 5000

# Portanto, o investimento mais apropriado é o cíclico, pois se obtem o maior lucro esperado.

In [50]:
# Questão 3

# O desempenho do Random Forest depende do equilíbrio entre a força individual das árvores e a correlação entre elas. 
# Árvores fortes aumentam a qualidade das decisões, enquanto baixa correlação permite que os erros de uma árvore sejam
# compensados pelas outras, resultando em um modelo mais robusto.